Read DF's

In [ ]:
import numpy as np
import pandas as pd

us_trending_df = pd.read_csv("./data/stage_1_us_trending.csv")
color_text_df = pd.read_csv("./data/stage_2_thumbnail_colors_and_text.csv")
face_df = pd.read_csv("./data/stage_2_thumbnail_face.csv")
objects_df = pd.read_csv("./data/stage_2_thumbnail_objects_vectors.csv")

Merge DF's

In [ ]:
df_all = pd.concat([us_trending_df, color_text_df, face_df, objects_df], axis=1)
df_labeled_only = df_all[df_all.categoryId != 0.0]
df_labeled_only.head()

In [ ]:
corr_matrix = df_labeled_only.corr(method='spearman')

target_attr = 'categoryId'
corr_attr_threshold = 0.8
drop_set = set()

corr_matrix_filtered = corr_matrix[(abs(corr_matrix) > corr_attr_threshold) & (corr_matrix < 1.0)]

for k in corr_matrix_filtered:
    dct = dict(corr_matrix_filtered[k].dropna())

    for k2 in dct:

        if abs(corr_matrix[k][target_attr]) > abs(corr_matrix[k2][target_attr]):
            drop_set.add(k2)
            print(k,k2)
        else:
            drop_set.add(k)
            print(k2,k)

# for col in drop_set:
#     df.drop(col, axis=1, inplace=True)

print(drop_set)

#jakis test hi2 powinien być zamiast korelacji

Calculate correlations between attributes and target class

In [ ]:
from matplotlib import pyplot as plt

corr_matrix_filtered = dict(df_labeled_only.corr(method='spearman')[target_attr].drop(target_attr).dropna())
corr_matrix_filtered = dict(sorted(corr_matrix_filtered.items(), key=lambda item: item[1], reverse=True))

plt.scatter(range(len(corr_matrix_filtered)), corr_matrix_filtered.values())
# jakiś wrapper można spróbować na grupach atrybutów

Find attributes with small correlation

In [ ]:
corr_target_threshold = 0.2

for k in list(corr_matrix_filtered.keys()):
    if abs(corr_matrix_filtered[k]) < corr_target_threshold:
        corr_matrix_filtered.pop(k)

plt.scatter(range(len(corr_matrix_filtered)), corr_matrix_filtered.values())

Drop attributes with small correlation from data and save to csv

In [ ]:
columns_to_drop = [ele for ele in list(df_labeled_only.columns) if ele not in list(corr_matrix_filtered.keys())]
df_labeled_only.drop(columns=columns_to_drop, axis=1, inplace=True)
df_labeled_only.to_csv("./data/stage_3_feature_importance.csv", index=False)
df_labeled_only.head()